In [1]:
!pip install datasets seacrowd bitsandbytes
from transformers import AutoTokenizer, Phi3ForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━

In [2]:
!pip install -U bitsandbytes
# Load pre-trained model and tokenizer
checkpoint = "microsoft/Phi-3.5-mini-instruct"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)
model = Phi3ForCausalLM.from_pretrained(
    checkpoint,
    torch_dtype=torch.bfloat16,  # Use bfloat16 for memory efficiency
    trust_remote_code=True,
    device_map="auto",  # Automatically map the model to available GPUs
    load_in_8bit=True,
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

In [3]:
!pip install -U peft bitsandbytes
from peft import LoraConfig, get_peft_model, TaskType
# Define LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["qkv_proj", "o_proj"],  # Adjust based on your model architecture
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_2_SEQ_LM",
    # bnb_8bit=True  # Enable bitsandbytes 8-bit quantization
)
# Apply LoRA to the model
model = get_peft_model(model, lora_config)

In [4]:
# 1. Memuat Dataset
dataset = load_dataset("SEACrowd/liputan6")
print("Dataset loaded:", dataset)
train_dataset = dataset["train"].shuffle(seed=42).select(range(100))
print("Train dataset:", train_dataset)
val_dataset = dataset["validation"].shuffle(seed=42).select(range(100))
print("Validation dataset:", val_dataset)
test_dataset = dataset["test"].shuffle(seed=42).select(range(100))
print("Test dataset:", test_dataset)

README.md:   0%|          | 0.00/3.23k [00:00<?, ?B/s]

liputan6.py:   0%|          | 0.00/7.52k [00:00<?, ?B/s]

The repository for SEACrowd/liputan6 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/SEACrowd/liputan6.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset loaded: DatasetDict({
    train: Dataset({
        features: ['document', 'id', 'summary'],
        num_rows: 193883
    })
    test: Dataset({
        features: ['document', 'id', 'summary'],
        num_rows: 10972
    })
    validation: Dataset({
        features: ['document', 'id', 'summary'],
        num_rows: 10972
    })
})
Train dataset: Dataset({
    features: ['document', 'id', 'summary'],
    num_rows: 100
})
Validation dataset: Dataset({
    features: ['document', 'id', 'summary'],
    num_rows: 100
})
Test dataset: Dataset({
    features: ['document', 'id', 'summary'],
    num_rows: 100
})


In [31]:
print(test_dataset[1])

{'document': 'Liputan6 . com , Jakarta : Kejaksaan Agung akan memanggil mantan Presiden B . J . Habibie . Namun , kepastian pemanggilan itu tergantung pada pemeriksaan Tim Penyidik Kejagung dalam kasus penggelapan dana nonbujeter Badan Urusan Logistik sebesar Rp 54 , 6 miliar . Demikian dikemukakan Kepala Pusat Penerangan dan Hukum Kejagung Moeljohardjo di Jakarta , Jumat ( 09/11 ) [ baca : Mantan Presiden Habibie Akan Dipanggil Kejagung ] . Selain itu , Moeljohardjo juga kembali akan memanggil mantan Deputi Keuangan Bulog Ruskandar dan pengelola kontraktor yang menyalurkan sembako Wilfred Simatupang sebagai saksi . Keduanya mangkir hadir untuk memenuhi panggilan Kejagung , Kamis kemarin . Menurut Moeljohardjo keduanya diduga mengetahui aliran penggelapan dana Bulog Rp 40 miliar dengan tersangka mantan Ketua Bulog Rahardi Ramelan . ( YYT/Lita Hariyani , Hendi Bernadi dan Dwi Guntoro ) .', 'id': '23428', 'summary': 'Mantan Presiden B . J . Habibie akan diperiksa Kejaksaan Agung dalam ka

In [5]:
from datasets import load_dataset
!pip install datasets
# Preprocess the dataset for summarization tasks
def preprocess_function(examples):
    inputs = examples["document"]
    targets = examples["summary"]
    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding="max_length",
    )
    # Tokenize the labels
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=128,
            truncation=True,
            padding="max_length",
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize the dataset
train_tokenized_datasets = train_dataset.map(preprocess_function, batched=True)
val_tokenized_datasets = val_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [6]:
!pip install trl
from peft import (
        get_peft_model,
        prepare_model_for_kbit_training,
        LoraConfig
    )
from trl import SFTTrainer
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 16.9 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/peft/mapping.py:185: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'microsoft/Phi-3.5-mini-instruct' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(


In [9]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./phi3-liputan6-summarizer-lora",  # Directory for saving model checkpoints
    evaluation_strategy="epoch",
    learning_rate=5e-4,  # Higher learning rate for LoRA fine-tuning
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,  # Limit number of saved checkpoints
    logging_dir="./logs",
    logging_steps=100,
    fp16=True,  # Enable mixed precision for faster training
    push_to_hub=False,
)

# Define the Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_tokenized_datasets,
    eval_dataset=val_tokenized_datasets,
    peft_config=lora_config,
    formatting_func=formatting_func,
    # max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_args
)

# Train the model with LoRA
trainer.train()





/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-9-057f443dae99>:27: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:359: UserWarning: You passed a dataset that is already processed (contains an `input_ids` field) together with a valid formatting function. Therefore `formatting_func` will be ignored. Either remove the `formatting_func` or pass a dataset that is not already processed.
  warning

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Epoch,Training Loss,Validation Loss
1,No log,2.106869
2,No log,2.101843
3,No log,2.118138


/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


TrainOutput(global_step=75, training_loss=2.0722650146484374, metrics={'train_runtime': 562.3285, 'train_samples_per_second': 0.533, 'train_steps_per_second': 0.133, 'total_flos': 3435077409177600.0, 'train_loss': 2.0722650146484374, 'epoch': 3.0})

In [34]:
test_dataset = dataset["test"]
print("Test dataset:", test_dataset)
print(train_dataset[0])

Test dataset: Dataset({
    features: ['document', 'id', 'summary'],
    num_rows: 10972
})
{'document': 'Liputan6 . com , Jombang : Maman Sugianto alias Sugik , yang masih berstatus terdakwa kasus pembunuhan Asrori , ditangguhkan penahanannya sejak Senin ( 24/11 ) malam . Penangguhan penahanannya ini dimanfaatkan Sugik untuk berkunjung ke rumah sanak saudaranya . Kedatangan Sugik disambut penuh haru bahkan sang bibi langsung jatuh pingsan . Sugik bebas bersyarat dari Lembaga Pemasyarakatan Jombang setelah menjadi korban salah tangkap dan peradilan sesat dalam kasus pembunuhan Asrori . Terdakwa lainnya yakni Devid dan Kemat , hingga kini masih dipenjara dan tengah mengajukan peninjauan kembali ( PK ) ke Mahkamah Agung . Sugik dan istrinya berharap majelis hakim lebih bijaksana untuk membebaskan dirinya dari semua tuduhan . Sebab , ia memang merasa tak membunuh Asrori seperti yang dituduhkan polisi dan jaksa [ baca : Saksi Ahli : Terdakwa Bisa Dibebaskan ] . ( UPI/Bambang Ronggo ) .', '

In [39]:
from peft import PeftModel

peft_model_id = "/content/phi3-liputan6-summarizer-lora/checkpoint-75"
peft_model = PeftModel.from_pretrained(model, peft_model_id, torch_dtype=torch.float16, offload_folder="lora_results/lora_7/temp")
def generate_prompt(artikel, output=None, eos_token="</s>"):
  instruction = """
rigkas artikel berita dibawah ini:
'document': 'Liputan6 . com , Jombang : Maman Sugianto alias Sugik , yang masih berstatus terdakwa kasus pembunuhan Asrori , ditangguhkan penahanannya sejak Senin ( 24/11 ) malam . Penangguhan penahanannya ini dimanfaatkan Sugik untuk berkunjung ke rumah sanak saudaranya . Kedatangan Sugik disambut penuh haru bahkan sang bibi langsung jatuh pingsan . Sugik bebas bersyarat dari Lembaga Pemasyarakatan Jombang setelah menjadi korban salah tangkap dan peradilan sesat dalam kasus pembunuhan Asrori . Terdakwa lainnya yakni Devid dan Kemat , hingga kini masih dipenjara dan tengah mengajukan peninjauan kembali ( PK ) ke Mahkamah Agung . Sugik dan istrinya berharap majelis hakim lebih bijaksana untuk membebaskan dirinya dari semua tuduhan . Sebab , ia memang merasa tak membunuh Asrori seperti yang dituduhkan polisi dan jaksa [ baca : Saksi Ahli : Terdakwa Bisa Dibebaskan ] . ( UPI/Bambang Ronggo ) .'
'summary': 'Maman Sugianto yang masih berstatus terdakwa kasus pembunuhan Asrori , ditangguhkan penahanannya sejak Senin malam . Kesempatan ini dimanfaatkan Sugik untuk berkunjung ke rumah sanak saudaranya .'

Artikel """
  input = f"{artikel}\n"
  output = f"""Summary:
#answer:
{output + ' ' + eos_token if output else ''} """
  prompt = (" ").join([instruction, input, output])
  return prompt
input_prompt = generate_prompt(test_dataset[1]["document"])
input_tokens = tokenizer(input_prompt, return_tensors="pt")["input_ids"].to("cuda")
with torch.amp.autocast('cuda'):
    generation_output = peft_model.generate(
        input_ids=input_tokens,
        max_new_tokens=100,
        do_sample=True,
        top_k=10,
        top_p=0.9,
        temperature=0.3,
        repetition_penalty=1.15,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
    )
output_text = tokenizer.decode(generation_output[0], skip_special_tokens=True)
translated_text = ""

print("bismnillah",output_text)




bismnillah 
rigkas artikel berita dibawah ini:

'document': 'Liputan6 . com , Jombang : Maman Sugianto alias Sugik , yang masih berstatus terdakwa kasus pembunuhan Asrori , ditangguhkan penahanannya sejak Senin ( 24/11 ) malam . Penangguhan penahanannya ini dimanfaatkan Sugik untuk berkunjung ke rumah sanak saudaranya . Kedatangan Sugik disambut penuh haru bahkan sang bibi langsung jatuh pingsan . Sugik bebas bersyarat dari Lembaga Pemasyarakatan Jombang setelah menjadi korban salah tangkap dan peradilan sesat dalam kasus pembunuhan Asrori . Terdakwa lainnya yakni Devid dan Kemat , hingga kini masih dipenjara dan tengah mengajukan peninjauan kembali ( PK ) ke Mahkamah Agung . Sugik dan istrinya berharap majelis hakim lebih bijaksana untuk membebaskan dirinya dari semua tuduhan . Sebab , ia memang merasa tak membunuh Asrori seperti yang dituduhkan polisi dan jaksa [ baca : Saksi Ahli : Terdakwa Bisa Dibebaskan ] . ( UPI/Bambang Ronggo ) .'
'summary': 'Maman Sugianto yang masih berstatus

In [50]:
# Calculate BLEU score for this sentence
bleu_score = sentence_bleu([test_dataset[1]['summary']], output_text)

# Print the results
print(f"Input: {test_dataset[1]['document']}")
print(f"Predicted: {output_text}")
print(f"Reference: {test_dataset[1]['summary']}")
print(f"BLEU Score: {bleu_score:.4f}")
print("-" * 50)




Input: Liputan6 . com , Bandar Lampung : Sebanyak 51 anak di bawah umur lima tahun terserang busung lapar atau marasmus karena kekurangan gizi di Kota Madya Bandar Lampung . Lima di antaranya tewas . Data tersebut diungkapkan Kepala Dinas Kesehatan Kota Bandar Lampung M . Sudarman , baru-baru ini . Menurut Sudarman , Dinas Kesehatan Bandar Lampung mencatat sekitar 51 anak terserang busung lapar yang tersebar di beberapa kecamatan , selama periode 1999 sampai 2001 . Kebanyakan anak penderita busung tersebut berasal dari keluarga yang hidup di bawah garis kemiskinan . Selain kekurangan gizi , komplikasi radang paru-paru juga menjadi satu faktor penyebab kematian anak penderita busung lapar tersebut . Data Dinas Kesehatan menunjukkan pada 1999 , ditemukan 41 anak terserang penyakit busung lapar . Sebagian besar penderita berdomisili di kampung miskin Umbul Kunci . Jumlah penderita busung lapar menurun pada 2000 , yakni hanya sembilan anak . Sedangkan September 2001 , seorang anak meningga